### 5. Setup

In [1]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 7982
WEBHOOK_BASE = ""
ADMIN_URL = "http://the_wallet-agent:7981"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL)


IPython autoawait is `on`, and set to use `asyncio`


In [2]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)


### 6. Check existing credential

In [3]:
credential_id = "Operation 0"
credential = await agent_controller.credentials.get_by_id(credential_id)
print(credential)

{'referent': 'Operation 0', 'attrs': {'speed': '1', 'dose': '1', 'volt': '1', 'type': 'drug_B'}, 'schema_id': 'PEcmuSQ7G3z771J7dse1R5:2:Operation:0.0.3', 'cred_def_id': 'PEcmuSQ7G3z771J7dse1R5:3:CL:48:default', 'rev_reg_id': None, 'cred_rev_id': None}


### 7. Check connection

In [4]:
response = await agent_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 1:
    connection = response['results'][1]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
else:
    print("You must create a connection")
    

Results :  [{'their_label': 'HOSPITAL', 'state': 'active', 'connection_id': '719fea26-a7ba-494c-9bc3-b7e89f552fda', 'created_at': '2023-01-05 10:58:40.422736Z', 'updated_at': '2023-01-05 10:58:55.829434Z', 'routing_state': 'none', 'rfc23_state': 'completed', 'accept': 'manual', 'their_role': 'invitee', 'invitation_mode': 'once', 'their_did': '6d6WJ9Wkgg7yxrRwzLEjTS', 'my_did': '7G7GLRFp7itdyrAn7ojJgs', 'invitation_key': '7btLbSrWD19PnAq5zWpnkB9ey6MhC8LGmzDMKzCrjDJP'}, {'their_label': 'MEDICAL_DEVICE_COMPUTER', 'state': 'active', 'connection_id': '8494d36d-b4dd-431d-b90d-f9371db7c56f', 'created_at': '2023-01-05 10:59:12.673705Z', 'updated_at': '2023-01-05 10:59:33.046513Z', 'routing_state': 'none', 'rfc23_state': 'completed', 'accept': 'manual', 'their_role': 'invitee', 'invitation_mode': 'once', 'their_did': 'ANrdVkJN9zaxJANC319kcm', 'my_did': 'N94AQp8pxv1DL2qwnVoxBr', 'invitation_key': '4ftHuzAW3pcct3uSo2mncWLLdPNEs1CaQgLN5uSAREds'}]
Connection : {'their_label': 'MEDICAL_DEVICE_COMPUT

### 8. Go to Medical_device_computer notebook to send verify request

### 9. Send verify response

In [5]:
response = await agent_controller.proofs.get_records()
print(response)

print('\n')

state = response['results'][0]["state"]
presentation_exchange_id = response['results'][0]['presentation_exchange_id']
presentation_request = response['results'][0]['presentation_request']

print('Presentation Exchange ID\n')
print(response['results'][0]['presentation_exchange_id'])
print('Presentation Request Object\n')
print(response['results'][0]['presentation_request'])
print('Requested Attributes\n')
print(response['results'][0]['presentation_request']['requested_attributes'])
requested_attribs = response['results'][0]['presentation_request']['requested_attributes']
print('Requested Predicates\n')
print(response['results'][0]['presentation_request']['requested_predicates'])
requested_predicates = response['results'][0]['presentation_request']['requested_predicates']

{'results': [{'presentation_request_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation', '@id': '08b24ffa-159d-4b47-8b6c-be62c956b825', 'request_presentations~attach': [{'@id': 'libindy-request-presentation-0', 'mime-type': 'application/json', 'data': {'base64': 'eyJuYW1lIjogIlByb29mIG9mIENvbXBsZXRpb24gb2YgUHlEZW50aXR5IFNTSSBUdXRvcmlhbCIsICJ2ZXJzaW9uIjogIjEuMCIsICJyZXF1ZXN0ZWRfYXR0cmlidXRlcyI6IHsiMF90eXBlX3V1aWQiOiB7Im5hbWUiOiAidHlwZSIsICJyZXN0cmljdGlvbnMiOiBbeyJpc3N1ZXJfZGlkIjogIlBFY211U1E3RzN6NzcxSjdkc2UxUjUifV19LCAiMF9kb3NlX3V1aWQiOiB7Im5hbWUiOiAiZG9zZSIsICJyZXN0cmljdGlvbnMiOiBbeyJpc3N1ZXJfZGlkIjogIlBFY211U1E3RzN6NzcxSjdkc2UxUjUifV19LCAiMF9zcGVlZF91dWlkIjogeyJuYW1lIjogInNwZWVkIiwgInJlc3RyaWN0aW9ucyI6IFt7Imlzc3Vlcl9kaWQiOiAiUEVjbXVTUTdHM3o3NzFKN2RzZTFSNSJ9XX0sICIwX3ZvbHRfdXVpZCI6IHsibmFtZSI6ICJ2b2x0IiwgInJlc3RyaWN0aW9ucyI6IFt7Imlzc3Vlcl9kaWQiOiAiUEVjbXVTUTdHM3o3NzFKN2RzZTFSNSJ9XX19LCAicmVxdWVzdGVkX3ByZWRpY2F0ZXMiOiB7fSwgIm5vbmNlIjogIjY3NTg1OD

### 10. Find credential

In [6]:
if state == "request_received":
    print(
    "Received Request -> Query for credentials in the wallet that satisfy the proof request")

# include self-attested attributes (not included in credentials)
credentials_by_reft = {}
revealed = {}
self_attested = {}
predicates = {}

# select credentials to provide for the proof
credentials = await agent_controller.proofs.get_presentation_credentials(presentation_exchange_id)
print(credentials)

if credentials:
    for row in sorted(
        credentials,
        key=lambda c: dict(c["cred_info"]["attrs"]),
        reverse=True,
    ):
        for referent in row["presentation_referents"]:
            if referent not in credentials_by_reft:
                credentials_by_reft[referent] = row

for referent in presentation_request["requested_attributes"]:
    if referent in credentials_by_reft:
        revealed[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ],
            "revealed": True,
        }
    else:
        self_attested[referent] = "South Africa"

for referent in presentation_request["requested_predicates"]:
    if referent in credentials_by_reft:
        predicates[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ]
        }

print("\nGenerate the proof")
proof = {
    "requested_predicates": predicates,
    "requested_attributes": revealed,
    "self_attested_attributes": self_attested,
}
print(proof)
print("\nXXX")
print(predicates)
print(revealed)
print(self_attested)

Received Request -> Query for credentials in the wallet that satisfy the proof request
[{'cred_info': {'referent': 'Operation 0', 'attrs': {'speed': '1', 'volt': '1', 'type': 'drug_B', 'dose': '1'}, 'schema_id': 'PEcmuSQ7G3z771J7dse1R5:2:Operation:0.0.3', 'cred_def_id': 'PEcmuSQ7G3z771J7dse1R5:3:CL:48:default', 'rev_reg_id': None, 'cred_rev_id': None}, 'interval': None, 'presentation_referents': ['0_volt_uuid', '0_type_uuid', '0_speed_uuid', '0_dose_uuid']}]

Generate the proof
{'requested_predicates': {}, 'requested_attributes': {'0_type_uuid': {'cred_id': 'Operation 0', 'revealed': True}, '0_dose_uuid': {'cred_id': 'Operation 0', 'revealed': True}, '0_speed_uuid': {'cred_id': 'Operation 0', 'revealed': True}, '0_volt_uuid': {'cred_id': 'Operation 0', 'revealed': True}}, 'self_attested_attributes': {}}

XXX
{}
{'0_type_uuid': {'cred_id': 'Operation 0', 'revealed': True}, '0_dose_uuid': {'cred_id': 'Operation 0', 'revealed': True}, '0_speed_uuid': {'cred_id': 'Operation 0', 'revealed':

### 11. Send proof back

In [7]:
response = await agent_controller.proofs.send_presentation(presentation_exchange_id, proof)
print(response)

{'presentation_request_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation', '@id': '08b24ffa-159d-4b47-8b6c-be62c956b825', 'request_presentations~attach': [{'@id': 'libindy-request-presentation-0', 'mime-type': 'application/json', 'data': {'base64': 'eyJuYW1lIjogIlByb29mIG9mIENvbXBsZXRpb24gb2YgUHlEZW50aXR5IFNTSSBUdXRvcmlhbCIsICJ2ZXJzaW9uIjogIjEuMCIsICJyZXF1ZXN0ZWRfYXR0cmlidXRlcyI6IHsiMF90eXBlX3V1aWQiOiB7Im5hbWUiOiAidHlwZSIsICJyZXN0cmljdGlvbnMiOiBbeyJpc3N1ZXJfZGlkIjogIlBFY211U1E3RzN6NzcxSjdkc2UxUjUifV19LCAiMF9kb3NlX3V1aWQiOiB7Im5hbWUiOiAiZG9zZSIsICJyZXN0cmljdGlvbnMiOiBbeyJpc3N1ZXJfZGlkIjogIlBFY211U1E3RzN6NzcxSjdkc2UxUjUifV19LCAiMF9zcGVlZF91dWlkIjogeyJuYW1lIjogInNwZWVkIiwgInJlc3RyaWN0aW9ucyI6IFt7Imlzc3Vlcl9kaWQiOiAiUEVjbXVTUTdHM3o3NzFKN2RzZTFSNSJ9XX0sICIwX3ZvbHRfdXVpZCI6IHsibmFtZSI6ICJ2b2x0IiwgInJlc3RyaWN0aW9ucyI6IFt7Imlzc3Vlcl9kaWQiOiAiUEVjbXVTUTdHM3o3NzFKN2RzZTFSNSJ9XX19LCAicmVxdWVzdGVkX3ByZWRpY2F0ZXMiOiB7fSwgIm5vbmNlIjogIjY3NTg1ODczNjU4MTIzMzY

### **Terminate**

In [10]:
response = await agent_controller.terminate()
print(response)

None
